# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [2]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeCaseConfig.from_file(config_path)

post_proc_cfg

CeCaseConfig(time_scale_conversion=TimeScaleParameters(CST_real=0.56, CST_sim=16.89), shape_coefficient={'measurement_1': CeConfig(statistics=[BasicStatisticModel(stats='mean'), BasicStatisticModel(stats='rms'), BasicStatisticModel(stats='skewness'), BasicStatisticModel(stats='kurtosis'), ParameterizedStatisticModel(stats='mean_eq', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', params=ExtremeAbsoluteParamsModel(method_type='Absolute')), ParameterizedStatisticModel(stats='max', params=ExtremePeakParamsModel(method_type='Peak', peak_factor=3.0))], zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['t1_ym'], exclude=['p1_xp'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Process shape coefficient use case

In [4]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import CeOutput, process_Ce

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.h5")

for cfg_label, cfg in post_proc_cfg.shape_coefficient.items():
    logger.info(f"Processing {cfg_label} ...")

    Ce_output: CeOutput = process_Ce(
        mesh=mesh,
        cfg=cfg,
        cp_path=cp_data_path,
        time_scale_factor=post_proc_cfg.time_scale_conversion.time_scale
    )

    # Ce_output.save_outputs(file_lbl="Ce", cfg_label=cfg_label, path_manager=path_manager)

[2024-05-17 09:46:54,428] [INFO] - cfdmod - Processing measurement_1 ... (2689420235.py:7)
[2024-05-17 09:46:54,429] [INFO] - cfdmod - Getting geometry data... (Ce_data.py:140)
[2024-05-17 09:46:54,443] [DEBUG] - cfdmod - Surface p1_xp ignored! (Ce_geom.py:52)
[2024-05-17 09:46:54,473] [INFO] - cfdmod - Tabulating geometry data... (Ce_data.py:143)
        region_idx  point_idx       area           n_x       n_y       n_z
0    303-lanternim          0   0.379758  1.000000e+00 -0.000000  0.000000
1    303-lanternim          1   0.562488  1.000000e+00  0.000000  0.000000
2    303-lanternim          2   0.112028  1.000000e+00  0.000000  0.000000
3    303-lanternim          3   0.098361  1.000000e+00  0.000000  0.000000
4    303-lanternim          4   0.641173  1.000000e+00  0.000000  0.000000
..             ...        ...        ...           ...       ...       ...
557       44-t2_yp       2910  24.822884  3.445718e-07  0.024955  0.999689
558       61-t2_yp       2911  31.682699 -1.251417